In [1]:
! pip install --quiet google-ads environs psycopg2-binary

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
from google.ads.google_ads.client import GoogleAdsClient
from google.ads.google_ads.errors import GoogleAdsException

client = GoogleAdsClient.load_from_storage('./google-ads.yaml')

In [9]:
# customer_id = '6521863227'
customer_id = '4607395644'

In [45]:
def create_conversion_action(client, customer_id, name):
    conversion_action_service = client.get_service(
        "ConversionActionService", version="v6"
    )
    conversion_action_operation = client.get_type(
        "ConversionActionOperation", version="v6"
    )

    conversion_action = conversion_action_operation.create
    conversion_action.name = name
    conversion_action.type = client.get_type(
        "ConversionActionTypeEnum"
    ).UPLOAD_CLICKS
    conversion_action.category = client.get_type(
        "ConversionActionCategoryEnum"
    ).DEFAULT
    conversion_action.status = client.get_type(
        "ConversionActionStatusEnum"
    ).ENABLED

    # Add the conversion action.
    conversion_action_response = conversion_action_service.mutate_conversion_actions(
        customer_id, [conversion_action_operation]
    )
    
    return conversion_action_response

In [ ]:
create_conversion_action(client, customer_id, 'savvy_investor')

In [95]:
ga_service = client.get_service("GoogleAdsService", version="v6")

query = f"""
SELECT click_view.gclid
FROM click_view
WHERE segments.date DURING YESTERDAY
"""

response = ga_service.search_stream(customer_id, query=query)
[row for batch in response for row in batch.results]

[]

In [ ]:
from datetime import datetime

def get_conversion_action(client, customer_id, name):
    ga_service = client.get_service("GoogleAdsService", version="v6")

    query = f"""
      SELECT conversion_action.id, conversion_action.name
      FROM conversion_action
      WHERE conversion_action.name = '{name}'
    """

    response = ga_service.search_stream(customer_id, query=query)
    try:
        row = next(row for batch in response for row in batch.results)
        return row.conversion_action
    except StopIteration:
        return None

def create_click_conversion(client, customer_id, action_name, gclid, utc_dt):
    ca = get_conversion_action(client, customer_id, action_name)
    if ca is None:
        raise Exception(f"Could not find conversion_action with name {action_name}")

    click_conversion = client.get_type("ClickConversion", version="v6")
    click_conversion.conversion_action = ca.resource_name    
    click_conversion.gclid = gclid
    click_conversion.conversion_date_time = utc_dt.strftime("%Y-%m-%d %H:%M:%S+00:00")        

    conversion_upload_service = client.get_service(
        "ConversionUploadService", version="v6"
    )

    conversion_upload_response = conversion_upload_service.upload_click_conversions(
        customer_id, [click_conversion], partial_failure=True
    )
    return conversion_upload_response



create_click_conversion(client, customer_id, 'savvy_investor', 'CKSDxc_qhLkCFQyk4AodO24Arg', datetime.utcnow())

In [ ]:
datetime.utcnow().isoformat()

In [ ]:
dir(datetime.utcnow())

In [68]:
datetime.utcnow().strftime("yyyy-mm-dd hh:mm:ss+|-hh:mm")

'yyyy-mm-dd hh:mm:ss+|-hh:mm'

In [13]:
from environs import Env
import psycopg2

env = Env()
env.read_env('.env', override=True)

conn = psycopg2.connect(
    dbname=env("PG_DATABASE"),
    user=env("PG_USER"),
    host=env("PG_HOST"),
    port=env("PG_PORT"),
    password=env("PG_PASSWORD"),
)

In [14]:
from consumer import Consumer
from report_conversions import report_conversions
from google.ads.google_ads.client import GoogleAdsClient

consumer = Consumer(conn, "signups_ga", "event_consumer_groups", 5)

work = lambda o, n: report_conversions(conn, client, customer_id, o, n)


In [ ]:
consumer.do_work(work)

partial_failure_error {
  code: 3
  message: "This Google Click ID could not be decoded., at conversions[0].gclid"
  details {
    type_url: "type.googleapis.com/google.ads.googleads.v6.errors.GoogleAdsFailure"
    value: "\nT\n\003\370\006\003\022*This Google Click ID could not be decoded.\032\005*\003qux\"\032\022\017\n\013conversions\030\000\022\007\n\005gclid"
  }
}
results {
}

